In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
df = pd.read_csv('tv_shows_final.csv')
#Let's change the 100/100 votes to 99/100 for practical reasons
df = df.replace('100/100','99/100')
#Check for Tv series that are not available through Netflix, Prime Video or Disney+. This 'extra' series are going to be separated from the rest.
extra = df.loc[(df['Netflix']== 0) & (df['Prime Video']== 0) & (df['Disney+']== 0)]

In [3]:
#A dataframe with all the types of Tv series' genres
genres = df['genres'].unique().tolist()
genres = DataFrame (genres, columns=['Genres'])

In [4]:
#There are a lot of different genres. So in order for the user to know the genres to look for, we are going to sample 15 of them as reference.
genres.sample(15)

,Genres
179,"Action,Comedy,History"
137,"Comedy,Horror,Sci-Fi"
231,"Adventure,Fantasy,War"
28,"Adventure,Comedy,Crime"
106,Game-Show
380,"Action,Fantasy,Sci-Fi"
273,"Adventure,Documentary,Reality-TV"
76,"Comedy,Mystery"
323,"Adventure,Family,Fantasy"
56,"Action,Adventure,Sci-Fi"


In [5]:
genre = input('What kind of genre are you interested in? [Warning: case sensitive] ')

What kind of genre are you interested in? [Warning: case sensitive]  Drama


In [6]:
filter = df['genres'].str.contains(genre)
df = df[filter]

In [7]:
#No musical!
filter = ~df['genres'].str.contains('Musical')
df = df[filter]

In [8]:
#IMDB and Rotten Tomatoes' votes must be float type
df['IMDb']=df['IMDb'].str[0:3]
df['IMDb']=df['IMDb'].astype(float)
df['Rotten Tomatoes']=df['Rotten Tomatoes'].str[0:2]
df['Rotten Tomatoes']=df['Rotten Tomatoes'].astype(int)
df['Rotten Tomatoes'] /=10

In [9]:
#The rating column is an average of RT and IMDB votes
df['Rating'] = (df['IMDb']+df['Rotten Tomatoes'])/2
df['Rating'] = df['Rating'].round(2)

In [10]:
#The only columns that matters are Title and Rating
col = ['Title', 'Rating']

In [11]:
#Netflix
Netflix_df=df.loc[df['Netflix']==1]
Netflix_df = Netflix_df[col]
Netflix_df.sort_values(by='Rating', ascending=False, inplace=True)

In [12]:
#Prime Video
PV_df=df.loc[df['Prime Video']==1]
PV_df = PV_df[col]
PV_df.sort_values(by='Rating', ascending=False, inplace=True)

In [13]:
#Disney+
Disney_df=df.loc[df['Disney+']==1]
Disney_df = Disney_df[col]
Disney_df.sort_values(by='Rating', ascending=False, inplace=True)

In [14]:
#Now for the 'extra' dataframe:

In [15]:
filter = extra['genres'].str.contains(genre)
extra = extra[filter]

filter = ~extra['genres'].str.contains('Musical')
extra = extra[filter]

extra['IMDb']=extra['IMDb'].str[0:3]
extra['IMDb']=extra['IMDb'].astype(float)
extra['Rotten Tomatoes']=extra['Rotten Tomatoes'].str[0:2]
extra['Rotten Tomatoes']=extra['Rotten Tomatoes'].astype(int)
extra['Rotten Tomatoes'] /=10

extra['Rating'] = (extra['IMDb']+extra['Rotten Tomatoes'])/2
extra['Rating'] = extra['Rating'].round(2)
col = ['Title', 'Rating']

Extra_df = extra[col]
Extra_df.sort_values(by='Rating', ascending=False, inplace=True)

<ipython-input-15-c645d3e049c6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Extra_df.sort_values(by='Rating', ascending=False, inplace=True)


In [16]:
#The result of the analysis is going to be saved as an excel file. The file name is the same as the genre been chosen.

In [18]:
with pd.ExcelWriter(f'{genre}.xlsx') as file:
    Netflix_df.to_excel(file, sheet_name='Netflix', index=False) #Sheet for Netflix Series
    PV_df.to_excel(file, sheet_name='Prime Video', index=False) #Sheet for Prime Video Series
    Disney_df.to_excel(file, sheet_name='Disney +', index=False) #Sheet for Disney+ Series
    Extra_df.to_excel(file, sheet_name='Extra', index=False) #Sheet for the 'Extra' Series